In [1]:
# JupyterLab에서 django shell을 실행시키기 위한 설정
import os
import django

os.environ["DJANGO_SETTINGS_MODULE"] = "config.settings" # config/settings.py 모듈 지정
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

In [61]:
from polls.models import Question, Choice

c = Choice.objects.get(pk=1)
print(c)

1. 고양이


In [62]:
# Model Manager -> Model클래스.objects => SQL을 실행하는 메소드들 제공. (DB작업 관리자)
print(type(Choice.objects), type(Question.objects))

<class 'django.db.models.manager.Manager'> <class 'django.db.models.manager.Manager'>


In [63]:
result = Choice.objects.all() # select * from tb(choice)
print(type(result))

<class 'django.db.models.query.QuerySet'>


In [64]:
# QuerySet: iterable
for r in result:
    print(r)

1. 고양이
2. 토끼
3. 귤
4. 딸기
5. 바퀴벌레
6. 곱등이
7. 흰색
8. 초록색


In [65]:
# QuerySet: subscriptable
result[0], len(result)

(<Choice: 1. 고양이>, 8)

# 조회
- 'Model.objects.all() : 전체조회
- 'Model.objects.filter(), exclude(), get() 
   - where절이 있는 조회

In [8]:
result_list = Question.objects.all()
print("조회결과 개수:", len(result_list))
print("실행된 SQL문조회 - QuerySet.query")
print(result_list.query)

조회결과 개수: 4
실행된 SQL문조회 - QuerySet.query
SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [9]:
for result in result_list:
    # print(result) # model.__str__()
    # 개별 Field 조회(instance 변수)
    print(result.id, result.pk, end=", ") # id-primarykey field이름, pk: primary key
    print(result.question_text, result.pub_date)


1 1, 좋아하는 동물은 무엇인가요? 2025-01-06 01:16:39.250787+00:00
2 2, 좋아하는 과일은 무엇입니까? 2025-01-06 01:16:50.689864+00:00
4 4, 싫어하는 곤충은 무엇입니까? 2025-01-07 00:06:49.642304+00:00
5 5, 좋아하는 색은 무엇입니까? 2025-01-07 00:17:23.758325+00:00


In [10]:
# 조회결과에서 개별 데이터 조회 - indexing
r = result_list[1]
print(r.pk, r.question_text, r.pub_date.strftime("%Y-%m-%d %H:%M:%S"))

2 좋아하는 과일은 무엇입니까? 2025-01-06 01:16:50


In [11]:
# QuerySet은 음수 index는 지원하지 않는다
result_list[-1]

ValueError: Negative indexing is not supported.

In [12]:
result_list[:1] # slicing -반환타입 -> list

[<Question: 1. 좋아하는 동물은 무엇인가요?>]

In [13]:
# QuerySet.first() : 첫번째 조회값 . result_list[0]
# QuerySet.last() : 마지막 조회값 . result_list[-1]
result_list.first()
result_list.last()

<Question: 5. 좋아하는 색은 무엇입니까?>

In [16]:
## 정렬 - sql: orderby 절
# queryset.order_by("field명") : asc
# queryset.order_by("-field명") : desc
## 기준 컬럼이 여러개일 경우 나열한다

# result_list = Question.objects.all()
result_list = Question.objects.all().order_by("-pk") # id desc
result_list = Question.objects.all().order_by("question_text") # question_text asc
for result in result_list: 
    print(result.pk, result.question_text, result.pub_date)

4 싫어하는 곤충은 무엇입니까? 2025-01-07 00:06:49.642304+00:00
2 좋아하는 과일은 무엇입니까? 2025-01-06 01:16:50.689864+00:00
1 좋아하는 동물은 무엇인가요? 2025-01-06 01:16:39.250787+00:00
5 좋아하는 색은 무엇입니까? 2025-01-07 00:17:23.758325+00:00


In [59]:
result_list = Choice.objects.all().order_by("-votes", "choice_text")
# SQL: orderby votes desc, choice_text asc
for result in result_list:
    print(result.pk, result.choice_text, result.votes)

1 고양이 11
2 토끼 11
4 딸기 10
3 귤 8
7 흰색 5
8 초록색 2
6 곱등이 1
5 바퀴벌레 1


In [18]:
print(result_list.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" ORDER BY "polls_choice"."votes" DESC, "polls_choice"."choice_text" ASC


## where (조건)으로 조회
- `filter()`: 조회기간이 True행동을 조회 -> QuerySet return
- `exclude()`: 조회기간이 False행동을 조회 -> QuerySet return 
- `get()`: 조회결과가 1개월일 때만 사용(pk=값), 조회결과가 없거나 2개 이상이면 Exception발생 -> Model return
- 조회조건 형식: `filed이름__연산자 = 값`

In [19]:
result =  Question.objects.filter(pk=1)  # pk = 1
result =  Question.objects.get(pk=1)     # pk = 1
result =  Question.objects.exclude(pk=1) # where not (pk = 1)
print(type(result))
result

<class 'django.db.models.query.QuerySet'>


<QuerySet [<Question: 2. 좋아하는 과일은 무엇입니까?>, <Question: 4. 싫어하는 곤충은 무엇입니까?>, <Question: 5. 좋아하는 색은 무엇입니까?>]>

In [20]:
# 비교연산
result = Choice.objects.filter(pk__lt = 5) # where pk < 5
result = Choice.objects.filter(pk__lte = 5) # where pk <= 5
result = Choice.objects.filter(pk__gt = 15) # where pk > 15
result = Choice.objects.filter(pk__gte = 15) # where pk >= 15
result = Choice.objects.filter(choice_text = "뱀") # get은 하나만
print(result.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."votes", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."choice_text" = 뱀


In [21]:
# 문자열 컬럼 부분일치 (like)
result = Question.objects.filter(question_text__contains = "색은")
# where question_text like "%색은%"

result = Question.objects.filter(question_text__startswith = "좋아하는")
# where question_text like "좋아하는%"

result = Question.objects.filter(question_text__endswith = "무엇입니까?")
# where question_text like "%무엇입니까?"

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\'
2. 좋아하는 과일은 무엇입니까?
4. 싫어하는 곤충은 무엇입니까?
5. 좋아하는 색은 무엇입니까?


In [22]:
# where 컬럼 in [v1, v2, ..] 연산자
Question.objects.filter(pk__in = [1, 3, 4])
Question.objects.exclude(pk__in = [1, 3, 4]) # not

<QuerySet [<Question: 2. 좋아하는 과일은 무엇입니까?>, <Question: 5. 좋아하는 색은 무엇입니까?>]>

In [58]:
# where 컬럼 between A and B
Choice.objects.filter(pk__range = [0, 2]) # pk between 0 and 2

<QuerySet [<Choice: 1. 고양이>, <Choice: 2. 토끼>]>

### where 절의 and, or
- `AND`: 조건을 나열한다
- `OR`: 각 조건을 Q()함수에 넣고 | 로 연결한다

In [25]:
# and
result = Question.objects.filter(
    question_text__endswith = "무엇입니까?",
    pk__lt = 3
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE ("polls_question"."id" < 3 AND "polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\')
2. 좋아하는 과일은 무엇입니까?


In [26]:
# or
from django.db.models import Q

result = Question.objects.filter(
    Q(question_text__endswith = "무엇입니까?") | Q(pk__gte = 3)
)

# ~Q(조건) : not 조건
## AND나 OR로 조회조건을 합칠 때 개별 조건에 NOT을 붙이려면 Q()를 이용용
result = Question.objects.filter(
    ~Q(question_text__endswith = "무엇입니까?") | Q(pk__gte = 3)
)

print(result.query)
for r in result:
    print(r)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE (NOT ("polls_question"."question_text" LIKE %무엇입니까? ESCAPE '\') OR "polls_question"."id" >= 3)
1. 좋아하는 동물은 무엇인가요?
4. 싫어하는 곤충은 무엇입니까?
5. 좋아하는 색은 무엇입니까?


### 조회 컬럼을 지정
- `values(컬럼명, ..)`
    - 개별(행) 조회 결과를 dictionary로 반환

In [27]:
result = Question.objects.all().values('pk', 'question_text')
result.query

In [28]:
result = Question.objects.filter(pk__lt = 3).values('pk', 'question_text')
result

<QuerySet [{'pk': 1, 'question_text': '좋아하는 동물은 무엇인가요?'}, {'pk': 2, 'question_text': '좋아하는 과일은 무엇입니까?'}]>

In [31]:
try:
    a = Question.objects.get(pk=20)
except:
    print('조회결과가 없습니다.')

조회결과가 없습니다.


### 집계함수
- aggregate(집계함수(컬럼명), ..) : 전체 행 기준 집계
- values('기준컬럼').annotate(집계함수(컬럼명), ..) : group by 후 집계

In [32]:
from django.db.models import Count, Sum, Avg, Min, Max, StdDev, Variance

result = Question.objects.aggregate(
    # Count("id")   # select count(id) from ..
    cnt=Count("id") # 변수명을 key값을 사용
)
print(type(result)) # dict. 기본 key 패턴: field명__집계함수이름
result#['id__count']

<class 'dict'>


{'cnt': 4}

In [34]:
# 한번에 여러 집계
result = Choice.objects.aggregate(
    cnt=Count('id'),
    vote_min=Min('votes'), 
    vote_max=Max('votes') 
)
print(type(result))
result

<class 'dict'>


{'cnt': 8, 'vote_min': 1, 'vote_max': 11}

In [35]:
# 집계함수를 이용한 연산. 변수명(key) = 연산식
## select max(votes) -  min(votes) as min

Choice.objects.aggregate(min_max_gap = (Max('votes')-Min('votes')))

{'min_max_gap': 10}

In [36]:
Choice.objects.aggregate(
    Count('votes'), Sum('votes'), Avg('votes'),  Min('votes'), Max('votes'), StdDev('votes'), Variance('votes')
)

{'votes__count': 8,
 'votes__sum': 49,
 'votes__avg': 6.125,
 'votes__min': 1,
 'votes__max': 11,
 'votes__stddev': 4.136348026943574,
 'votes__variance': 17.109375}

In [37]:
# select sum(votes)->annotate() from choice group by question (values())

result = Choice.objects.values('question').annotate(Sum('votes'))
result

<QuerySet [{'question': 1, 'votes__sum': 22}, {'question': 2, 'votes__sum': 18}, {'question': 4, 'votes__sum': 2}, {'question': 5, 'votes__sum': 7}]>

In [38]:
result = Choice.objects.values('question').annotate(Min('votes'), Max('votes'))
result
## group별 결과를 dictionary에 담아서 QuerySet으로 묶어 반환.

<QuerySet [{'question': 1, 'votes__min': 11, 'votes__max': 11}, {'question': 2, 'votes__min': 8, 'votes__max': 10}, {'question': 4, 'votes__min': 1, 'votes__max': 1}, {'question': 5, 'votes__min': 2, 'votes__max': 5}]>

# JOIN
- 부모테이블: question, 자식테이블: choice

In [56]:
# 부모테이블: Question, 자식테이블: Choice
c1 = Choice.objects.get(pk=1)
c1

<Choice: 1. 고양이>

In [57]:
print(c1.pk, c1.votes, c1.choice_text)
print(c1.question, type(c1.question), c1.question_id)
print(c1.question.question_text)

1 11 고양이
1. 좋아하는 동물은 무엇인가요? <class 'polls.models.Question'> 1
좋아하는 동물은 무엇인가요?


In [88]:
result_list = Choice.objects.filter(choice_text__endswith='색')
for result in result_list:
    print(
        result.choice_text, # 보기 텍스트
        result.question.question_text  # 자식객체.ForeignKey필드 ->참조하는 부모데이터
    )

* 부모레이블 기준으로 자식레이블을 조회
    - 부모 model_instance.자식모델클래스이름_set: 자식 객체를 조회

In [48]:
Question.objects.all()

<QuerySet [<Question: 1. 좋아하는 동물은 무엇인가요?>, <Question: 2. 좋아하는 과일은 무엇입니까?>, <Question: 4. 싫어하는 곤충은 무엇입니까?>, <Question: 5. 좋아하는 색은 무엇입니까?>]>

In [50]:
q1 = Question.objects.get(pk=5)
q1

<Question: 5. 좋아하는 색은 무엇입니까?>

In [51]:
q1.choice_set
# choice(Choice클래스) -> q1을 참조하는 Choice의 행들을 조회할 수 있는 manager를 반환 -> RelateManager

<django.db.models.fields.related_descriptors.create_reverse_many_to_one_manager.<locals>.RelatedManager at 0x29e836b8980>

In [53]:
# 설문 문제 - 보기
question_list = Question.objects.all()
for question in question_list:
    print(f"{question.pk}. {question.question_text}")
    for idx, choice in enumerate(question.choice_set.all()):
        print(f"\t{idx+1}. {choice.choice_text}: {choice.votes}")

1. 좋아하는 동물은 무엇인가요?
	1. 고양이: 11
	2. 토끼: 11
2. 좋아하는 과일은 무엇입니까?
	1. 귤: 8
	2. 딸기: 10
4. 싫어하는 곤충은 무엇입니까?
	1. 바퀴벌레: 1
	2. 곱등이: 1
5. 좋아하는 색은 무엇입니까?
	1. 흰색: 5
	2. 초록색: 2


# Insert/Update
 - model객체.save()

In [66]:
# insert
new_question = Question(question_text="새질문입니다.")
pk = new_question.save() # 저장되 있지 않은 pk의 instance를 save -> insert
print(pk)

None


In [67]:
print(new_question.pk) # insert 후에 자동 등록된 값들(pk, pub_date)를 모델객체에 넣어준다
print(new_question.pub_date)

6
2025-01-07 00:33:01.562216+00:00


In [71]:
# 조회
q = Question.objects.get(pk=6)
# update
q.question_text = "여행가고 싶은 곳이 어디인가요?" #pk=3인 행의 값을 변경
q.save() # DB에 있는 pk 행의 값을 수정 -> update
# save(): 반환값 object - update시 pk값 | None - insert

In [72]:
Question.objects.get(pk=6)

<Question: 6. 여행가고 싶은 곳이 어디인가요?>

In [73]:
# delete
del_question = Question(pk=6) # 삭제할 primary key를 가진 객체를 생성
del_question.pk, del_question.question_text
del_question.delete()

(1, {'polls.Question': 1})

In [74]:
Question.objects.all()

<QuerySet [<Question: 1. 좋아하는 동물은 무엇인가요?>, <Question: 2. 좋아하는 과일은 무엇입니까?>, <Question: 4. 싫어하는 곤충은 무엇입니까?>, <Question: 5. 좋아하는 색은 무엇입니까?>]>

In [75]:
# ORM -> 조건에 데이터를 조회한 후에 삭제
print(Choice.objects.all().count()) # QuerySet.count() 조회데이터수 (len(QuerySet)과 동일)
print(Choice.objects.filter(votes__gt=50).count())

8
0


In [76]:
# choice에서 votes가 50 이상인 행들을 삭제
## delete from choice where votes >= 50
del_list = Choice.objects.filter(votes__gt=50)
for del_obj in del_list:
    print(del_obj.pk, del_obj.votes)
    del_obj.delete()